In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# import mne
# import numpy as np
# from copy import deepcopy
# import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet import Simulation
from esinet import Net
from esinet.forward import create_forward_model, get_info
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
plot_params = dict(surface='white', hemi='both', verbose=0)

## Create Forward model
First we create a template forward model which comes with the esinet package

In [3]:
info = get_info(sfreq=100, kind="biosemi64")
fwd = create_forward_model(sampling="ico3", info=info)
leadfield = fwd["sol"]["data"]
n_chans, n_dipoles = leadfield.shape

import mne
from scipy.sparse.csgraph import laplacian

adjacency = mne.spatial_src_adjacency(fwd['src']).toarray()
laplace_operator = abs(laplacian(adjacency))
# laplace_operator = laplace_operator @ laplace_operator
# laplace_operator = np.identity(n_dipoles)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    2.0s remaining:    2.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.3s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    2.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


-- number of adjacent vertices : 1284


[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:    0.2s remaining:    0.2s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.2s finished


# Models

## The Generator

In [24]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, InputLayer, Input, Lambda, LayerNormalization
from tensorflow.keras.layers import Activation, Dropout, ActivityRegularization, TimeDistributed, Reshape, Permute, GaussianNoise, add

import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
# import tensorflow_probability as tfp
from tensorflow.keras.regularizers import l1, l2, l1_l2

def weightedLoss(w):
    def loss(true, pred):
        print("DISC", true, pred)
        
        error = K.square(true - pred)
        # error = K.abs(true - pred)
        error = K.switch(K.equal(true, 0), w * error , error)
        # normalize for number of active sites
        error = error / tf.linalg.norm(true, axis=-1)
        return error
    return loss

def weightedLossGan(w):
    def loss(true, pred):
        print("GAN", true, pred)
        error = K.square(true - pred)
        # error = K.abs(true - pred)
        error = K.switch(K.equal(true, 0), w * error , error)
        # normalize for number of active sites
        error = error * tf.linalg.norm(true)
        return -error
    return loss


def custom_gan_loss(y_ones, y_both):
    y_true = y_both[0]
    y_pred = y_both[1]
    # print(y_ones, y_true, y_pred)
    
    error = -tf.keras.losses.CosineSimilarity()(y_true, y_pred)
    # error = -tf.keras.losses.mean_squared_error(y_true, y_pred)
    # blur =  tf.cast(tf.size(y_true), dtype=tf.float32) / tf.math.count_nonzero(y_true, dtype=tf.float32)
    # blur = K.mean(K.abs(y_true))
    # error = K.square(y_true -y_pred)
    # error = error / tf.linalg.norm(y_true, axis=-1)
    # error = -K.mean(error)
    # normalize for number of active sites
    # error = error / tf.linalg.norm(y_true)
    # norm = tf.linalg.norm(y_true)

    # Maximize variance of norms
    # norm_variance = 1/tf.math.reduce_std(tf.linalg.norm(y_true, axis=-1))

    return error #+ norm

def square(x):
    return K.square(x) * K.sign(x)
    
def scale_act(x):
    return tf.transpose(tf.transpose(x) / tf.math.reduce_max(K.abs(x), axis=-1))
        

class CustomDropout(tf.keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training: #you can remove this line,, so that you can use dropout on inference time
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

def define_gan(g_model, d_model, latent_dim, learning_rate=0.0001):
    n_chans, n_dipoles = leadfield.shape
    leadfield_ = tf.cast(leadfield, dtype=tf.float32)
    d_model.trainable = False
    inputs = tf.keras.Input(shape=(latent_dim), name='Input_Generator')
    output_1 = g_model(inputs)
    eeg = Lambda(lambda x: tf.transpose(tf.linalg.matmul(leadfield_, tf.transpose(x))), output_shape=(None, n_chans))(output_1)
    
    print("eeg from source from g: ", eeg)
    eeg_normed = LayerNormalization()(eeg)
    
    # print("eeg_normed from source from g: ", eeg_normed)
    output_3 = d_model(eeg_normed)
    model = Model(inputs, [output_1, output_3])

    # Add loss to model
    loss = K.abs(tf.keras.losses.CosineSimilarity()(output_1, output_3)) #+ l1_sparsity(output_1)
    model.add_loss(loss)

    # Compile
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    
    return model


def define_discriminator(n_dense_layers=2, hidden_units=200, learning_rate=0.0001):
    input_shape = (n_chans)
    inputs = tf.keras.Input(shape=input_shape, name='Input_Discriminator')
    activation = "tanh"
    leadfield_ = tf.cast(leadfield, dtype=tf.float32)
    fc = Dense(hidden_units, activation=activation, name="HL_D1")(inputs)
    for i in range(n_dense_layers-1):
        fc = Dense(hidden_units, activation=activation, name=f"HL_D{i+2}")(fc)
    out = Dense(n_dipoles, activation="linear", name="Output_Final")(fc)

    # fc1 = Dense(hidden_units, activation=activation, name="HL_D1")(inputs)
    # fc2 = Dense(hidden_units, activation=activation, name="HL_D2")(fc1)
    # adder = add([fc1, fc2])

    # fc3 = Dense(hidden_units, activation=activation, name="HL_D3")(adder)
    # fc4 = Dense(hidden_units, activation=activation, name="HL_D4")(fc3)
    # adder = add([fc3, fc4])
    # out = Dense(n_dipoles, activation="linear", name="Output_Final")(adder)
    
   
    
    model = tf.keras.Model(inputs=inputs, outputs=out, name='Discriminator')
    model.add_loss(l1_sparsity(out))
    # model.add_loss( tf.keras.losses.CosineSimilarity(name="meas_loss")(tf.linalg.matmul(leadfield_, tf.transpose(out)),  tf.transpose(inputs)) )

    model.compile(loss=tf.keras.losses.CosineSimilarity(), optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    return model
  
def define_generator(latent_dim=100):
    laplace_operator_ = tf.cast(laplace_operator, dtype=tf.float32)

    lrelu = tf.keras.layers.LeakyReLU()
    inputs = tf.keras.Input(shape=(latent_dim), name='Input_Generator')
    fc = Dense(latent_dim, activation=lrelu, name="HL_G1")(inputs)
    gen_out_sparse = Dense(n_dipoles, name="Output_Generator", activation="tanh")(fc)
    
    # fc1 = Dense(hidden_units, activation=lrelu, name="HL_D1")(inputs)
    # fc2 = Dense(hidden_units, activation=lrelu, name="HL_D2")(fc1)
    # adder = add([fc1, fc2])

    # fc3 = Dense(hidden_units, activation=lrelu, name="HL_D3")(adder)
    # fc4 = Dense(hidden_units, activation=lrelu, name="HL_D4")(fc3)
    # adder = add([fc3, fc4])
    # gen_out_sparse = Dense(n_dipoles, name="Output_Generator", activation="tanh")(adder)

    gen_out = Lambda(lambda x: tf.transpose(tf.linalg.matmul(laplace_operator_, tf.transpose(x))), output_shape=(None, n_chans))(gen_out_sparse)
    model = tf.keras.Model(inputs=inputs, outputs=gen_out, name='Generator')
    model.add_loss(l1_sparsity( gen_out_sparse ))


    return model

def define_models(latent_dim, hidden_units=200, reg=1, n_dense_layers=2, activation="tanh", learning_rate=0.001):
    n_chans, n_dipoles = leadfield.shape
    leadfield_ = tf.cast(leadfield, dtype=tf.float32)
    laplace_operator_ = tf.cast(laplace_operator, dtype=tf.float32)
    lrelu = tf.keras.layers.LeakyReLU()

    # drop_min = 0.95
    # drop_max = 0.99
    # G MODEL
    # ------------------------------------------------------------------------
    inputs = tf.keras.Input(shape=(latent_dim), name='Input_Generator')
    fc = Dense(latent_dim, activation=lrelu, name="HL_G1")(inputs)
    # fc = GaussianNoise(0.5)(fc)

    fc = Dense(latent_dim, activation=lrelu, name="HL_G2")(fc)

    # gen_out = Dense(n_dipoles, name="Output_Generator", activation=activation, activity_regularizer=l1_l2(reg))(fc)
    gen_out = Dense(n_dipoles, name="Output_Generator", activation=activation)(fc)
    # gen_out = Dropout(0.5)(gen_out)

    # gen_out = ActivityRegularization(l2=reg)(gen_out)
    
    # gen_out = Dropout(0.97)(gen_out, training=True)
    # gen_out = Dropout(K.random_uniform((1,), drop_min, drop_max)[0])(gen_out, training=True)
    # Thresholding/ Sparsification
    # gen_out = Activation(scale_act)(gen_out)
    # gen_out = Activation("tanh")(gen_out)
    # gen_out = Lambda(lambda x: tf.cast(K.abs(x)==K.max(K.abs(x)), dtype=x.dtype) * x, output_shape=(None, n_dipoles))(gen_out)
    # gen_out = Lambda(lambda x: tf.cast(K.abs(x)>0.9, dtype=x.dtype) * x, output_shape=(None, n_dipoles))(gen_out)
    # gen_out = Lambda(lambda x: tf.cast(K.abs(x)>tfp.stats.percentile(K.abs(x), K.random_uniform((1,), drop_min, drop_max)), dtype=x.dtype) * x, output_shape=(None, n_dipoles))(gen_out)
    
    
    # Smoothing
    gen_out = Lambda(lambda x: tf.transpose(tf.linalg.matmul(laplace_operator_, tf.transpose(x))), output_shape=(None, n_chans))(gen_out)
    
    # Scaling
    # gen_out = LayerNormalization(center=False)(gen_out)
    gen_out = Activation(scale_act)(gen_out)
    gen_out = Activation("tanh")(gen_out)
    # Sparsify
    # gen_out = ActivityRegularization(l2=reg)(gen_out)
    g_model = tf.keras.Model(inputs=inputs, outputs=gen_out, name='Generator')
    g_model.compile(loss=custom_gan_loss, optimizer="adam")
    # g_model.build(input_shape=(latent_dim))
    
    # D MODEL
    # ------------------------------------------------------------------------
    input_shape = (n_chans)
    inputs2 = tf.keras.Input(shape=input_shape, name='Input_Discriminator')
    
    fc2 = Dense(hidden_units, activation=activation, name="HL_D1")(inputs2)
    for i in range(n_dense_layers-1):
        fc2 = Dense(hidden_units, activation=activation, name=f"HL_D{i+2}")(fc2)
        
    # out = Dense(n_dipoles, activation=activation, activity_regularizer=l2(reg), name="Output_Final")(fc2)
    out = Dense(n_dipoles, activation=activation, name="Output_Final")(fc2)
    # out = Lambda(lambda x: tf.transpose(tf.linalg.matmul(laplace_operator_, tf.transpose(x))), output_shape=(n_dipoles))(out)
    # out = Activation(scale_act)(out)
    # out = Activation("tanh")(out)
    # out = ActivityRegularization(l2=reg)(out)
    d_model = tf.keras.Model(inputs=inputs2, outputs=out, name='Discriminator')
    # d_model.add_loss(l1_sparsity(out))
    # d_model.compile(loss=tf.keras.losses.CosineSimilarity(), optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    d_model.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    
    # d_model.add_loss( tf.keras.losses.CosineSimilarity()(tf.linalg.matmul(leadfield_, tf.transpose(out)), tf.transpose(inputs2)) )
    # d_model.add_loss(norm_ratio_ineq(out))
    # d_model.build(input_shape=(latent_dim))
    
    # GAN MODEL
    # ------------------------------------------------------------------------
    d_model.trainable = False
    inputs = tf.keras.Input(shape=(latent_dim), name='Input_Generator')
    output_1 = g_model(inputs)
    
    print("source from g: ", output_1)
    eeg = Lambda(lambda x: tf.transpose(tf.linalg.matmul(leadfield_, tf.transpose(x))), output_shape=(None, n_chans))(output_1)
    
    print("eeg from source from g: ", eeg)
    eeg_normed = LayerNormalization()(eeg)
    
    # print("eeg_normed from source from g: ", eeg_normed)
    output_3 = d_model(eeg_normed)
    gan_model = Model(inputs, [output_1, output_3])

    # g_model.compile(loss=custom_gan_loss, optimizer="adam")
    
    # d_model.compile(loss=weightedLoss(10), optimizer="adam")
    
    # Construct your custom loss as a tensor
    print("OUTPUT1: ", output_1)
    print("OUTPUT3: ", output_3)
    
    # loss = -tf.keras.losses.MeanSquaredError()(output_1, output_3) #+ l1_sparsity(output_1)
    # loss = K.abs(tf.keras.losses.CosineSimilarity()(output_1, output_3))
    loss = -tf.keras.losses.CosineSimilarity()(output_1, output_3)
    
    # loss = tf.math.log(-tf.keras.losses.CosineSimilarity()(output_1, output_3)) + K.square(norm_ratio_ineq(output_1))/5
    # loss = batch_diversity(output_1) + l1_sparsity(output_1)
    # loss = norm_ratio_ineq(output_1)
    # loss = weightedLossGan(10)(output_1, output_3)
    
    # Add loss to model
    gan_model.add_loss(loss)

    gan_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    # gan_model.compile(loss=custom_gan_loss, optimizer="adam")
    
    return g_model, d_model, gan_model

def norm_ratio_ineq(x):
    l1 = K.mean(K.abs(x)) 
    l2 = K.mean(K.square(x)) 
    return l1 / l2 

def l1_sparsity(x):
    return K.mean(K.abs(x)) 

# def batch_diversity(x):
#     avg_source = K.mean(K.abs(x), axis=0)
#     l1 = K.mean(K.abs(avg_source)) 
#     l2 = K.mean(K.square(avg_source)) 
#     return l1 / l2

def batch_diversity(x):
    diversity = K.std(K.mean(K.abs(x), axis=0))
    return diversity

def prep_data(X, y):
    X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
    y = np.stack([(x / np.max(abs(x))) for x in y], axis=0)
    return X, y
    
def prep_data_sim(sim):
    X = np.squeeze(np.stack([eeg.average().data for eeg in sim.eeg_data]))
    X = np.stack([(x - np.mean(x)) / np.std(x) for x in X], axis=0)
    y = np.squeeze(np.stack([src.data for src in sim.source_data]))
    y = np.stack([(x / np.max(abs(x))) for x in y], axis=0)

    if len(X.shape) == 2:
        X = np.expand_dims(X, axis=-1)
        y = np.expand_dims(y, axis=-1)
    X = np.swapaxes(X, 1,2)
    y = np.swapaxes(y, 1,2)
    return X, y

def generate_samples(g_model, batch_size, latent_dim):
    x_input = np.random.randn(batch_size, latent_dim)
    sources = g_model.predict(x_input)
    return sources

n_epochs = 200
batch_size = 32
batch_number = 10
latent_dim = 16
g, d, gan = define_models(latent_dim, hidden_units=latent_dim)
# gan(np.random.randn(batch_size, latent_dim))

source from g:  KerasTensor(type_spec=TensorSpec(shape=(None, 1284), dtype=tf.float32, name=None), name='Generator/activation_11/Tanh:0', description="created by layer 'Generator'")
eeg from source from g:  KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='lambda_27/transpose_1:0', description="created by layer 'lambda_27'")
OUTPUT1:  KerasTensor(type_spec=TensorSpec(shape=(None, 1284), dtype=tf.float32, name=None), name='Generator/activation_11/Tanh:0', description="created by layer 'Generator'")
OUTPUT3:  KerasTensor(type_spec=TensorSpec(shape=(None, 1284), dtype=tf.float32, name=None), name='Discriminator/Output_Final/Tanh:0', description="created by layer 'Discriminator'")


# Simulate

In [5]:
settings = dict(duration_of_trial=0.01, extents=(1, 40), number_of_sources=(1,15), target_snr=1e99)
sim = Simulation(fwd, info, settings=settings).simulate(n_samples=100)
X_test = np.stack([eeg.average().data[:, 0] for eeg in sim.eeg_data], axis=0)
y_test = np.stack([source.data[:, 0] for source in sim.source_data], axis=0)
X_test, y_test = prep_data(X_test, y_test)

Simulating data based on sparse patches.


100%|██████████| 100/100 [00:00<00:00, 408.41it/s]


# Train

In [19]:
d_model.train_on_batch(X_history, y_history)

-1.4336764812469482

In [33]:
from tqdm.notebook import tqdm
n_chans, n_dipoles = leadfield.shape
n_epochs = 10000
batch_size = 64
latent_dim = 10
hidden_units = 400
disc_mod = 1
gen_mod = 1
reps = 10
learning_rate = 0.001
# g_model, d_model, gan_model = define_models(latent_dim, hidden_units=hidden_units, reg=reg, learning_rate=0.01)

d_model = define_discriminator(learning_rate=learning_rate)
g_model = define_generator(latent_dim=latent_dim)
gan_model = define_gan(g_model, d_model, latent_dim, learning_rate=learning_rate)

X_history = np.zeros((0, n_chans))
y_history = np.zeros((0, n_dipoles))
gan_losses = np.zeros(n_epochs)
d_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)
generated = []

for i in tqdm(range(n_epochs)):
    if i % disc_mod == 0:
        print("Train Disc")
        y = generate_samples(g_model, batch_size, latent_dim)
        X = (leadfield @ y.T).T
        X, _ = prep_data(X,y)
        generated.append(abs(y).mean(axis=0))
        X_history = np.append(X_history, X, axis=0)
        y_history = np.append(y_history, y, axis=0)
        idc = np.arange(X_history.shape[0])
        np.random.shuffle(idc)
        X_history = X_history[idc]
        y_history = y_history[idc]
        X_history = X_history[:batch_size]
        y_history = y_history[:batch_size]
        for _ in range(1):
            d_loss = d_model.train_on_batch(X_history, y_history)
        
        # d_loss = d_model.train_on_batch(X, y)
        d_model.trainable = True
        test_loss = d_model.evaluate(X_test, y_test, verbose=0)
        test_losses[i] = test_loss

        # print(f"disc_loss befoer: {d_loss}")
    if i % gen_mod == 0:# and d_loss<-0.15:
        print("Train Gen")
        x_input = np.random.randn(batch_size, latent_dim)

        X = np.ones((batch_size, n_dipoles))
        d_model.trainable = False
        gan_model.layers[4].trainable = False
        for _ in range(reps):
            gan_loss = gan_model.train_on_batch(x_input, X)
        # gan_loss = gan_model.train_on_batch(x_input, X)
        d_model.trainable = True
        gan_model.layers[4].trainable = True
      
    print(f'test_loss: {test_loss:.2f}, disc-loss: {d_loss:.2f}, gan-loss: {gan_loss:.2f}')
    d_losses[i] = d_loss
    
    gan_losses[i] = gan_loss

eeg from source from g:  KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='lambda_33/transpose_1:0', description="created by layer 'lambda_33'")


  0%|          | 0/10000 [00:00<?, ?it/s]

Train Disc
Train Gen
test_loss: 0.17, disc-loss: 0.18, gan-loss: 0.25
Train Disc
Train Gen
test_loss: 0.16, disc-loss: 0.07, gan-loss: 0.25
Train Disc
Train Gen
test_loss: 0.15, disc-loss: 0.08, gan-loss: 0.22
Train Disc
Train Gen
test_loss: 0.14, disc-loss: 0.06, gan-loss: 0.22
Train Disc
Train Gen
test_loss: 0.13, disc-loss: 0.05, gan-loss: 0.22
Train Disc
Train Gen
test_loss: 0.13, disc-loss: 0.02, gan-loss: 0.22
Train Disc
Train Gen
test_loss: 0.13, disc-loss: 0.04, gan-loss: 0.25
Train Disc
Train Gen
test_loss: 0.13, disc-loss: 0.05, gan-loss: 0.19
Train Disc
Train Gen
test_loss: 0.13, disc-loss: 0.05, gan-loss: 0.24
Train Disc
Train Gen
test_loss: 0.13, disc-loss: 0.06, gan-loss: 0.21
Train Disc
Train Gen
test_loss: 0.13, disc-loss: 0.06, gan-loss: 0.21
Train Disc
Train Gen
test_loss: 0.14, disc-loss: 0.06, gan-loss: 0.20
Train Disc
Train Gen
test_loss: 0.15, disc-loss: 0.04, gan-loss: 0.23
Train Disc
Train Gen
test_loss: 0.15, disc-loss: 0.10, gan-loss: 0.18
Train Disc
Train Gen

KeyboardInterrupt: 

# Train 2

In [489]:
from tqdm.notebook import tqdm
n_chans, n_dipoles = leadfield.shape
n_epochs = 10000
batch_size = 32
latent_dim = 200
hidden_units = 200
reg = 0.01
gen_mod = 1
disc_mod = 1
# g_model, d_model, gan_model = define_models(latent_dim, hidden_units=hidden_units, reg=reg, learning_rate=0.01)

d_model = define_discriminator()
g_model = define_generator(latent_dim=latent_dim)
gan_model = define_gan(g_model, d_model, latent_dim)

X_history = np.zeros((0, n_chans))
y_history = np.zeros((0, n_dipoles))
gan_losses = np.zeros(n_epochs)
d_losses = np.zeros(n_epochs)
test_losses = np.zeros(n_epochs)
generated = []

for i in tqdm(range(n_epochs)):
    

    if i % disc_mod == 0:
        print("Train Disc")
        y = generate_samples(g_model, batch_size, latent_dim)
        X = (leadfield @ y.T).T
        X, _ = prep_data(X,y)
        generated.append(abs(y).mean(axis=0))
        X_history = np.append(X_history, X, axis=0)
        y_history = np.append(y_history, y, axis=0)
        idc = np.arange(X_history.shape[0])
        np.random.shuffle(idc)
        X_history = X_history[idc]
        y_history = y_history[idc]
        X_history = X_history[:batch_size]
        y_history = y_history[:batch_size]
        d_loss = d_model.train_on_batch(X_history, y_history)
        
        # d_loss = d_model.train_on_batch(X, y)
        d_model.trainable = True
        test_loss = d_model.evaluate(X_test, y_test, verbose=0)
        test_losses[i] = test_loss

        # print(f"disc_loss befoer: {d_loss}")
    if i % 1000 == 0:
        print("Spawn new generator")
        g_model = define_generator(latent_dim=latent_dim)
        gan_model = define_gan(g_model, d_model, latent_dim)
        for i in range(500):
            x_input = np.random.randn(batch_size, latent_dim)
            X = np.ones((batch_size, n_dipoles))
            d_model.trainable = False
            gan_model.layers[4].trainable = False
            gan_loss = gan_model.train_on_batch(x_input, X)
            d_model.trainable = True
            gan_model.layers[4].trainable = True
            print(f"new gan loss: {gan_loss}")


    print(f'test_loss: {test_loss:.2f}, disc-loss: {d_loss:.2f}, gan-loss: {gan_loss:.2f}')
    d_losses[i] = d_loss
    
    gan_losses[i] = gan_loss

eeg from source from g:  KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='lambda_362/transpose_1:0', description="created by layer 'lambda_362'")


  0%|          | 0/10000 [00:00<?, ?it/s]

Train Disc
Spawn new generator
eeg from source from g:  KerasTensor(type_spec=TensorSpec(shape=(None, 64), dtype=tf.float32, name=None), name='lambda_364/transpose_1:0', description="created by layer 'lambda_364'")
new gan loss: 0.4506118893623352
new gan loss: 0.4709557890892029
new gan loss: 0.4452674090862274
new gan loss: 0.4380176067352295
new gan loss: 0.4492713212966919
new gan loss: 0.4417171776294708
new gan loss: 0.4613807797431946
new gan loss: 0.4543728232383728
new gan loss: 0.43639203906059265
new gan loss: 0.41760823130607605
new gan loss: 0.4414896070957184
new gan loss: 0.42913782596588135
new gan loss: 0.43249356746673584
new gan loss: 0.44499266147613525
new gan loss: 0.41836971044540405
new gan loss: 0.41693076491355896
new gan loss: 0.4224749207496643
new gan loss: 0.43969854712486267
new gan loss: 0.42927825450897217
new gan loss: 0.4170489013195038
new gan loss: 0.42197248339653015
new gan loss: 0.40796566009521484
new gan loss: 0.4163704216480255
new gan loss: 0

KeyboardInterrupt: 

# Tests

In [35]:
x_input = np.random.randn(batch_size, latent_dim)

X = np.ones((batch_size, n_dipoles))
d_model.trainable = False
gan_model.layers[4].trainable = False
for _ in range(reps):
    gan_loss = gan_model.train_on_batch(x_input, X)
# gan_loss = gan_model.train_on_batch(x_input, X)
d_model.trainable = True
gan_model.layers[4].trainable = True

stc_ = sim.source_data[0].copy()
noise = np.random.randn(1, latent_dim)
src = g_model(noise).numpy().T
eeg = leadfield @ src
eeg -= eeg.mean()
eeg /= eeg.std()
src_hat = d_model(eeg[np.newaxis]).numpy().T

print("MSE: ", np.mean((src-src_hat)**2), "Cos: ", tf.keras.losses.CosineSimilarity()(src, src_hat))


stc_.data = src
stc_.plot(**plot_params)

stc_.data = src_hat
stc_.plot(**plot_params)


MSE:  0.22771609 Cos:  tf.Tensor(-0.0529595, shape=(), dtype=float32)


In [24]:
stc = sim.source_data[0].copy()
stc_ = stc.copy()
stc_.data = np.stack(generated, axis=0).T
stc_.plot(**plot_params)

In [25]:
%matplotlib qt
plt.figure()
plt.plot(d_losses, label="Discriminator Loss")
plt.plot(gan_losses, label="GAN Loss")
plt.plot(test_losses, label="Test Loss")
plt.legend()

# # g_model, d_model, gan_model = define_models(latent_dim, hidden_units=hidden_units, reg=reg)

y = generate_samples(g_model, 1000, latent_dim)
data = abs(y).mean(axis=0)
stc_ = stc.copy()
stc_.data[:, 0] = data
stc_.plot(**plot_params)

stc_.data = y.T
stc_.plot(**plot_params)

plt.figure()
plt.hist(y.flatten())
print("Batch Diversity: ", batch_diversity(y).numpy(), "L1: ", l1_sparsity(y).numpy())

Batch Diversity:  0.39202377 L1:  1.9522592


Using control points [4.01885765 4.39036816 5.76854114]
Using control points [5.82157312 6.34829485 7.96443184]
Using control points [5.82157312 6.34829485 7.96443184]
Using control points [4.23217171 4.6658253  5.8257476 ]
Using control points [5.67898798 6.02882284 7.81410707]
Using control points [4.46965706 4.78317289 6.10744302]
Using control points [4.46965706 4.78317289 6.10744302]
Using control points [4.4028537  4.79486138 6.87534443]
Using control points [4.45385004 4.73737525 6.31460746]


# Evaluate

## Simulation

In [37]:
n_samples = 2
settings = dict(duration_of_trial=0.1, extents=(1,40), number_of_sources=(1,15), target_snr=1e99, source_number_weighting=False)
# settings = dict(duration_of_trial=0.1, extents=(1,40), number_of_sources=15, target_snr=1e99)
# settings = dict(duration_of_trial=0.01, extents=25, number_of_sources=3, target_snr=1e99)

sim_test = Simulation(fwd, info, settings=settings).simulate(n_samples=n_samples)

X = np.stack([eeg.average().data for eeg in sim_test.eeg_data], axis=0)
y = np.stack([src.data for src in sim_test.source_data], axis=0)

X, y = prep_data(X[0].T, y[0].T)

y_hat = d_model.predict(X)
y_hat.shape
stc = sim_test.source_data[0]
stc.data /= np.max(abs(stc.data))
stc.plot(**plot_params, brain_kwargs=dict(title="Ground Truth Sim"))

stc_hat = stc.copy()
stc_hat.data = y_hat.T
stc_hat.plot(**plot_params, brain_kwargs=dict(title="GAN"))
from scipy.stats import pearsonr
from esinet.evaluate import eval_auc
_, pos = util.unpack_fwd(fwd)[1:3]
auc = np.mean([np.mean(eval_auc(y_true, y_pred, pos)) for y_true, y_pred in zip(stc.data.T, stc_hat.data.T)])
cosine = tf.keras.losses.CosineSimilarity()(tf.cast(stc.data.T, dtype=tf.float32), tf.cast(stc_hat.data.T, dtype=tf.float32)).numpy()
r,p = pearsonr(stc.data.flatten(), stc_hat.data.flatten())
print(f'auc={auc:.2f}, cosine={cosine}, r={r:.2f}, p={p:.4f}\n')

Simulating data based on sparse patches.


100%|██████████| 2/2 [00:00<00:00, 143.25it/s]


auc=1.00, cosine=-0.09070241451263428, r=0.08, p=0.0000



Using control points [0.22290041 0.23714272 0.36731724]


In [69]:
sys.path.insert(0, '../../invert/')
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping
from invert.solvers.wrop import SolverLAURA
from invert.solvers.empirical_bayes import SolverChampagne

solver = SolverLAURA().make_inverse_operator(fwd)
# solver = SolverChampagne().make_inverse_operator(fwd)

stc_mne = solver.apply_inverse_operator(sim_test.eeg_data[0].average())
stc_mne.data = stc_mne.data / np.max(abs(stc_mne.data))
stc_mne.plot(**plot_params, brain_kwargs=dict(title=solver.name))
r,p = pearsonr(stc.data.flatten(), stc_mne.data.flatten())
auc = np.mean([np.mean(eval_auc(y_true, y_pred, pos)) for y_true, y_pred in zip(stc.data.T, stc_mne.data.T)])

print(f'auc={auc:.2f}, r={r:.2f}, p={p:.4f}\n')

auc=0.72, r=0.26, p=0.0000



Using control points [0.07413376 0.0919503  0.2329121 ]
Using control points [0.14399299 0.25143925 0.94449736]
Using control points [0.22729244 0.27810621 0.68943058]


# Test Discriminator with Generator

In [52]:
# generated data:
y = generate_samples(g_model, 32, latent_dim)
X = (leadfield @ y.T).T
X, y = prep_data(X,y)

y[np.isnan(y)] = 0
stc_hat.data = y.T
stc_hat.plot(**plot_params, brain_kwargs=dict(title="Ground Truth"), clim=dict(kind='value', pos_lims=(0, 0.5, 1)))

y_hat = d_model.predict(X)
y_hat[np.isnan(y_hat)] = 0

stc_hat.data = y_hat.T
stc_hat.plot(**plot_params, brain_kwargs=dict(title="GAN"), clim=dict(kind='value', pos_lims=(0, 0.5, 1)))

r,p = pearsonr(y.flatten(), y_hat.flatten())
auc = np.mean([np.mean(eval_auc(yy_true, yy_pred, pos)) for yy_true, yy_pred in zip(y, y_hat)])

print(f'auc={auc:.2f}, r={r:.2f}, p={p:.4f}\n')

auc=0.91, r=0.84, p=0.0000



# Linear Solution

In [53]:
sys.path.insert(0, '../../invert/')
from invert.solvers.minimum_norm_estimates import SolverDynamicStatisticalParametricMapping
from invert.solvers.wrop import SolverLAURA
from invert.solvers.empirical_bayes import SolverChampagne

evoked = mne.EvokedArray(X.T, info)

# solver = SolverLAURA().make_inverse_operator(fwd)
# solver = SolverChampagne().make_inverse_operator(fwd)

stc_mne = solver.apply_inverse_operator(evoked)
stc_mne.data = stc_mne.data / np.max(abs(stc_mne.data))
stc_mne.plot(**plot_params, brain_kwargs=dict(title=solver.name))
r,p = pearsonr(y.flatten(), stc_mne.data.flatten())
auc = np.mean([np.mean(eval_auc(yy_true, yy_pred, pos)) for yy_true, yy_pred in zip(y, stc_mne.data.T)])

print(f'auc={auc:.2f}, r={r:.2f}, p={p:.4f}\n')

auc=0.51, r=-0.01, p=0.0427



Using control points [0.02927872 0.03322611 0.06774781]
Using control points [0.42319688 0.4607384  0.93013235]


## old functions

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, InputLayer, Input, Lambda
import numpy as np


basic_sources = np.identity(n_dipoles)
basic_sources = laplace_operator @ basic_sources
basic_sources = np.concatenate([basic_sources, -1*basic_sources], axis=1)
basic_eeg = leadfield @ basic_sources
print(basic_sources.shape, basic_eeg.shape)

n_chans, n_dipoles = leadfield.shape

def define_generator(latent_dim):
    g_model = tf.keras.Sequential()
    input_shape = (None, latent_dim)
    g_model.add(InputLayer(input_shape=input_shape))
    g_model.add(Dense(latent_dim, name="HL1"))
    g_model.add(Dense(n_dipoles, name="Output"))
    # g_model.build()
    # g_model.compile(optimizer='adam', loss="mse")
    # g_model.summary()
    return g_model
    
def define_discriminator(hidden_units=100):
    input_shape = (None, n_chans)
    d_model = tf.keras.Sequential()
    d_model.add(InputLayer(input_shape=input_shape))
    d_model.add(Dense(hidden_units, name="HL1"))
    d_model.add(Dense(n_dipoles, name="Output"))
    d_model.build()
    d_model.compile(optimizer='adam', loss=tf.keras.losses.CosineSimilarity())
    # d_model.summary()
    return d_model

def define_gan(g_model, d_model, latent_dim):
    leadfield_ = tf.cast(leadfield, dtype=tf.float32)
    d_model.trainable = False
    
    input_shape = (None, latent_dim)
    
    lam = Lambda(lambda x: tf.transpose(tf.linalg.matmul(leadfield_, tf.transpose(x))), output_shape=(None, n_chans))(g_model.output)
    print(lam)
    discriminator = d_model(lam)
    model = tf.keras.Model(inputs=g_model.input, outputs=[d_model.output, g_model.output], name='Contextualizer')


    # model = tf.keras.Sequential()
    # model.add(g_model)
    # model.add(Lambda(lambda x: tf.linalg.matmul(leadfield_, x)))
    # model.add(d_model)
    # model.compile(loss='binary_crossentropy', optimizer="adam")

    return model
 

